<a href="https://colab.research.google.com/github/MarciaFG/scimobility/blob/main/LaplacianMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 800*800 Laplacian Matrix from Meso-Fields

* Calculate the Laplacian matrix of the square matrix.
* Calculate the eigenvalues and eigenvectors of the Laplacian matrix.
* Sort the eigenvalues in ascending order.
* The second smallest eigenvalue corresponds to the second eigenvector.
* Normalize the second eigenvector to have a unit length.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.
Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime → "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [ ]:
import numpy as np
import requests
import pandas as pd
from tqdm import tqdm
import torch
import nltk
import io
import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.colab import files
import humanize
!pip install 'networkx<2.7'
!pip install 'scipy>=1.8'
#!pip install google-sheets-to-csv
# enable data table format
from google.colab import data_table
data_table.enable_dataframe_formatter()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os.path
from google.colab import drive

# mount Google Drive to /content/drive/My Drive/
if os.path.isdir("/content/drive/My Drive"):
  print("Google Drive already mounted")
else:
  drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
id = '17RXABV5bp0-u_oSvKmKONW3f7FLzrjYl'
print (id) # Verify that you have everything
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('MesoFieldsMatrix.csv')  
edgelist = pd.read_csv('MesoFieldsMatrix.csv', sep=";") # Dataset is now stored in a Pandas Dataframe
edgelist.head(1)
print(edgelist.dtypes)

17RXABV5bp0-u_oSvKmKONW3f7FLzrjYl
source    int64
target    int64
weight    int64
dtype: object


In [ ]:
# concatenate the source and target columns to create a node list
nodelist = pd.concat([edgelist['source'], edgelist['target']]).unique()

# print the node list
print("Node list: ", nodelist)

Node list:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 24

In [ ]:
from networkx.linalg.graphmatrix import adjacency_matrix
import pandas as pd
import numpy as np
import networkx as nx

G=nx.from_pandas_edgelist(edgelist,'source','target','weight')

In [ ]:
Degrees = nx.degree(G, weight='weight')
Degrees 

DegreeView({0: 944790, 1: 1517891, 2: 363783, 3: 539938, 4: 2062685, 5: 906943, 6: 1153704, 7: 1110087, 8: 923040, 9: 568077, 10: 1455438, 11: 1355230, 12: 863706, 13: 593201, 14: 2107498, 15: 333538, 16: 580717, 17: 310783, 18: 991788, 19: 320679, 20: 2372674, 21: 415472, 22: 443867, 23: 1149114, 24: 1169857, 25: 983721, 26: 822165, 27: 416914, 28: 775295, 29: 685336, 30: 394767, 31: 1068060, 32: 1063226, 33: 992858, 34: 1122537, 35: 872075, 36: 396060, 37: 326509, 38: 736102, 39: 1165625, 40: 406580, 41: 937865, 42: 1095961, 43: 226039, 44: 365167, 45: 1878495, 46: 1079786, 47: 691174, 48: 690722, 49: 613522, 50: 460783, 51: 458855, 52: 549371, 53: 556691, 54: 842272, 55: 514160, 56: 1393325, 57: 700277, 58: 381854, 59: 317449, 60: 183572, 61: 1156091, 62: 1616585, 63: 1353485, 64: 654207, 65: 686809, 66: 745943, 67: 242738, 68: 209934, 69: 1395255, 70: 819937, 71: 384030, 72: 1149813, 73: 806983, 74: 452334, 75: 907093, 76: 729124, 77: 783817, 78: 362840, 79: 377174, 80: 560676, 81:

In [ ]:
# Compute the normalized Laplacian matrix
# The graph Laplacian is the matrix L = D - A, where A is the adjacency matrix and D is the diagonal matrix of node degrees.
laplacian_matrix = nx.laplacian_matrix(G, nodelist = nodelist, weight='weight').astype(float)

# Convert the Laplacian matrix to a dense numpy array and inspect
laplacian_matrix_array = laplacian_matrix.toarray()
df = pd.DataFrame(laplacian_matrix_array, columns=G.nodes(), index=G.nodes())
print(df.head(5))




        0          1         2         3          4       5       6    \
0  944790.0    -3990.0      -2.0     -75.0       -6.0   -28.0   -14.0   
1   -3990.0  1517891.0     -19.0     -86.0     -104.0   -79.0 -4067.0   
2      -2.0      -19.0  363783.0   -5065.0     -225.0  -325.0  -807.0   
3     -75.0      -86.0   -5065.0  539938.0     -526.0  -352.0  -782.0   
4      -6.0     -104.0    -225.0    -526.0  2062685.0 -2063.0  -354.0   

       7        8      9    ...    841   846  857     840   810    811   842  \
0     -3.0    -87.0 -127.0  ...   -4.0  -4.0  0.0     0.0   0.0   -1.0  -2.0   
1   -856.0  -3839.0  -44.0  ... -114.0 -51.0 -4.0   -79.0  -4.0    0.0 -12.0   
2  -2164.0    -95.0  -23.0  ...  -60.0  -3.0 -1.0 -1469.0 -25.0  -16.0  -1.0   
3  -5164.0   -300.0 -251.0  ...  -23.0  -3.0 -3.0   -32.0 -60.0 -116.0  -5.0   
4 -13158.0 -15833.0  -61.0  ...  -18.0  -4.0 -8.0   -88.0 -30.0    0.0 -16.0   

   843    830     844  
0  0.0   -7.0  -526.0  
1 -2.0   -5.0 -1621.0  
2 -2.0  

In [ ]:
ones = np.ones(865)
print(ones)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [ ]:
laplacian_matrix * ones

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
import scipy.sparse.linalg
# calculate the second eigenvalue of the Laplacian matrix

# compute all the eigenvalues and eigenvectors of the Laplacian matrix
eigenvalues, eigenvectors = scipy.linalg.eigh(laplacian_matrix.toarray())

# print the Laplacian eigenvalues
print("Laplacian eigenvalues: ", eigenvectors)

Laplacian eigenvalues:  [-4.64757642e-11  6.76135336e+02  9.99638239e+02  1.46845268e+03
  1.62084878e+03  1.64603282e+03  1.65277951e+03  1.67762752e+03
  1.70500192e+03  1.74146955e+03  1.75207132e+03  1.75875914e+03
  1.80884930e+03  1.84125343e+03  1.87625750e+03  1.91811793e+03
  1.96984208e+03  2.02769079e+03  2.03190796e+03  2.06154650e+03
  2.13555173e+03  2.16351386e+03  2.34642678e+03  2.35011424e+03
  2.35587733e+03  2.43070148e+03  2.44849990e+03  2.52330681e+03
  2.71351450e+03  2.72814308e+03  2.74364482e+03  2.75426974e+03
  2.81736341e+03  2.84649325e+03  2.90712153e+03  2.91690528e+03
  3.38749441e+03  3.40518370e+03  3.43115479e+03  3.52857381e+03
  3.53189696e+03  3.53395723e+03  3.54523743e+03  3.59440768e+03
  3.59811217e+03  3.60825510e+03  3.66113882e+03  3.69793606e+03
  3.77785212e+03  3.83176385e+03  3.87984455e+03  3.92634227e+03
  3.97923084e+03  3.98932159e+03  4.02191508e+03  4.05281155e+03
  4.13406744e+03  4.18616948e+03  4.20616420e+03  4.29915284e+03
 

In [ ]:
 second_eigenvector = eigenvectors[:, 1]

In [ ]:
laplacian_matrix * second_eigenvector
el = 7.82882617e-01 /second_eigenvector[0]
print(el)

676.1353364708476



2.   Find the first nonzero eigenvalue: To find the first nonzero eigenvalue of 
the Laplacian matrix, you can use the scipy.sparse.linalg.eigsh function with the which='SA' parameter to compute the smallest eigenvalues and eigenvectors of the matrix. The smallest nonzero eigenvalue should be the second smallest eigenvalue if the first eigenvalue is zero. Here's an example:

/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


TypeError: ignored

3. Sort eigenvalues and eigenvectors: Once you have the eigenvalues and eigenvectors of the Laplacian matrix, you can sort them in ascending order of eigenvalues to identify the most important eigenvectors. You can also normalize the eigenvectors to have unit length to make them easier to interpret. Here's an example:

In [ ]:
# sort the eigenvalues and eigenvectors in ascending order
sorted_indices = eigenvalues.argsort()
sorted_eigenvalues = eigenvalues[sorted_indices]
sorted_eigenvectors = eigenvectors[:, sorted_indices]

# print the sorted eigenvalues and eigenvectors
print("Sorted eigenvalues: ", sorted_eigenvalues)
print("Sorted eigenvectors: ", sorted_eigenvectors)

Sorted eigenvalues:  [2.47862122e-10 6.76135336e+02 9.99638239e+02]
Sorted eigenvectors:  [[-0.03400102 -0.00115788 -0.00116127]
 [-0.03400102 -0.00116607 -0.00113876]
 [-0.03400102 -0.00110961 -0.00116472]
 ...
 [-0.03400102 -0.00177458 -0.00102998]
 [-0.03400102 -0.00222576  0.00022728]
 [-0.03400102 -0.00129349 -0.00115702]]


How many zero eigenvalues are there?

In [ ]:
# count the number of nonzero eigenvalues
num_nonzero_eigenvalues = np.count_nonzero(sorted_eigenvalues)

# count the number of zero eigenvalues
num_zero_eigenvalues = len(sorted_eigenvalues) - num_nonzero_eigenvalues

# print the number of zero eigenvalues
print("Number of zero eigenvalues: ", num_zero_eigenvalues)
print("Number of nonzero eigenvalues: ", num_nonzero_eigenvalues)

Number of zero eigenvalues:  0
Number of nonzero eigenvalues:  3


In [ ]:
num_nonzero_eigenvalues = np.count_nonzero(sorted_eigenvalues)

# count the number of zero eigenvalues
num_zero_eigenvalues = len(sorted_eigenvalues) - num_nonzero_eigenvalues

# print the number of zero eigenvalues
print("Number of zero eigenvalues: ", num_zero_eigenvalues)

Number of zero eigenvalues:  0


In [ ]:
# print the shape of the Laplacian matrix
print("Laplacian matrix shape: ", laplacian_matrix.shape)

Laplacian matrix shape:  (865, 865)


In [ ]:
# sort the eigenvalues and eigenvectors in ascending order
sorted_indices = eigenvalues.argsort()
sorted_eigenvalues = eigenvalues[sorted_indices]
sorted_eigenvectors = eigenvectors[:, sorted_indices]

# print the sorted eigenvalues and eigenvectors
print("Sorted eigenvalues: ", sorted_eigenvalues)
print("Sorted eigenvectors: ", sorted_eigenvectors)

Sorted eigenvalues:  [2.47862122e-10 6.76135336e+02 9.99638239e+02]
Sorted eigenvectors:  [[-0.03400102 -0.00115788 -0.00116127]
 [-0.03400102 -0.00116607 -0.00113876]
 [-0.03400102 -0.00110961 -0.00116472]
 ...
 [-0.03400102 -0.00177458 -0.00102998]
 [-0.03400102 -0.00222576  0.00022728]
 [-0.03400102 -0.00129349 -0.00115702]]
